<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Examine-JSON-data-from-state_district_wise-API" data-toc-modified-id="Examine-JSON-data-from-state_district_wise-API-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Examine JSON data from <code>state_district_wise</code> API</a></span></li><li><span><a href="#Examine-district.csv" data-toc-modified-id="Examine-district.csv-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Examine <code>district.csv</code></a></span><ul class="toc-item"><li><span><a href="#Examine-district_wise.csv" data-toc-modified-id="Examine-district_wise.csv-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Examine <code>district_wise.csv</code></a></span></li></ul></li><li><span><a href="#Method-1:" data-toc-modified-id="Method-1:-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Method 1:</a></span></li><li><span><a href="#Method-2" data-toc-modified-id="Method-2-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Method 2</a></span></li><li><span><a href="#Method-3" data-toc-modified-id="Method-3-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Method 3</a></span></li></ul></div>

### Examine JSON data from `state_district_wise` API

This API gets the statistics for the current day. Latest confirmed cases for the day, and total cases from the beginning

In [22]:
import requests
import json
import pandas as pd

In [7]:
r = requests.get("https://api.covid19india.org/state_district_wise.json")

In [11]:
obj = json.loads(r.text)

In [83]:
# get info for MH
df = pd.DataFrame(obj["Maharashtra"]["districtData"])
df = df.T
df.drop(["notes"], inplace=True, axis=1)
df.columns = "total_" + df.columns

# put today's info in the df
res = pd.DataFrame(df.pop('total_delta').tolist())
res.index = df.index
df = df.join(res )

# change column names
cols = dict(((x, "today_" + x) for x in ["confirmed", "deceased", "recovered"]))
df.rename(columns=cols, inplace=True)

In [84]:
df

,total_active,total_confirmed,total_deceased,total_recovered,today_confirmed,today_deceased,today_recovered
Ahmednagar,3239,14274,172,10863,603,13,525
Akola,472,3347,143,2731,60,2,8
Amravati,1106,3879,101,2672,111,0,85
Aurangabad,6233,19500,579,12688,546,1,241
Beed,1968,3088,69,1051,263,2,90
Bhandara,237,631,9,385,43,4,9
Buldhana,883,2579,67,1629,72,0,38
Chandrapur,444,1146,9,693,65,0,36
Dhule,1473,5452,164,3813,88,3,39
Gadchiroli,111,553,1,441,13,0,5



### Examine `district.csv`

District-wise daily data from 26-Apr-2020

In [92]:
df1 = pd.read_csv("../districts.csv")

In [93]:
df1 = df1[(df1.State == "Maharashtra") & (df1.District == "Mumbai")]
df1

,Date,State,District,Confirmed,Recovered,Deceased,Other,Tested
192,2020-04-26,Maharashtra,Mumbai,5407,641,204,0,NaN
622,2020-04-27,Maharashtra,Mumbai,5776,711,219,0,NaN
1062,2020-04-28,Maharashtra,Mumbai,6169,798,244,0,NaN
1506,2020-04-29,Maharashtra,Mumbai,6644,936,270,0,NaN
1954,2020-04-30,Maharashtra,Mumbai,7061,1092,290,0,NaN
...,...,...,...,...,...,...,...,...
72908,2020-08-15,Maharashtra,Mumbai,127716,102749,7086,300,651593.0
73618,2020-08-16,Maharashtra,Mumbai,128726,103468,7133,300,656690.0
74328,2020-08-17,Maharashtra,Mumbai,129479,104301,7173,301,663400.0
75038,2020-08-18,Maharashtra,Mumbai,130410,105193,7222,302,663400.0


We see that the statistics for 19-Aug-2020 matches those of API. The daily increase(delta) can be calculated by finding difference between the rows

In [94]:
df1["new_confirmed"] = df1["Confirmed"].diff()

In [95]:
df1

,Date,State,District,Confirmed,Recovered,Deceased,Other,Tested,new_confirmed
192,2020-04-26,Maharashtra,Mumbai,5407,641,204,0,NaN,NaN
622,2020-04-27,Maharashtra,Mumbai,5776,711,219,0,NaN,369.0
1062,2020-04-28,Maharashtra,Mumbai,6169,798,244,0,NaN,393.0
1506,2020-04-29,Maharashtra,Mumbai,6644,936,270,0,NaN,475.0
1954,2020-04-30,Maharashtra,Mumbai,7061,1092,290,0,NaN,417.0
...,...,...,...,...,...,...,...,...,...
72908,2020-08-15,Maharashtra,Mumbai,127716,102749,7086,300,651593.0,-819.0
73618,2020-08-16,Maharashtra,Mumbai,128726,103468,7133,300,656690.0,1010.0
74328,2020-08-17,Maharashtra,Mumbai,129479,104301,7173,301,663400.0,753.0
75038,2020-08-18,Maharashtra,Mumbai,130410,105193,7222,302,663400.0,931.0


#### Examine `district_wise.csv`

In [108]:
df2 = pd.read_csv("../district_wise(1).csv")

In [109]:
df2 = df2[(df2.District == "Mumbai")]
df2

,SlNo,State_Code,State,District_Key,District,Confirmed,Active,Recovered,Deceased,Migrated_Other,Delta_Confirmed,Delta_Active,Delta_Recovered,Delta_Deceased,District_Notes,Last_Updated
346,385,MH,Maharashtra,MH_Mumbai,Mumbai,131542,17914,106057,7268,303,1132,222,864,46,NaN,NaN


In [111]:
df3 = pd.read_csv("../raw_data13.csv")

In [112]:
df3

,Entry_ID,State Patient Number,Date Announced,Age Bracket,Gender,Detected City,Detected District,Detected State,State code,Num Cases,Current Status,Contracted from which Patient (Suspected),Notes,Source_1,Source_2,Source_3,Nationality,Type of transmission,Status Change Date,Patient Number
0,218991,NaN,07/08/2020,NaN,NaN,NaN,Aizawl,Mizoram,MZ,2.0,Hospitalized,NaN,NaN,http://twitter.com/dipr_mizoram/status/1291547...,http://twitter.com/dipr_mizoram/status/1291547...,NaN,NaN,NaN,NaN,NaN
1,218992,NaN,07/08/2020,NaN,NaN,NaN,Adilabad,Telangana,TG,14.0,Hospitalized,NaN,As of 06/08/2020 (8 PM),https://t.me/indiacovid/12689,NaN,NaN,NaN,NaN,NaN,NaN
2,218993,NaN,07/08/2020,NaN,NaN,NaN,Bhadradri Kothagudem,Telangana,TG,82.0,Hospitalized,NaN,As of 06/08/2020 (8 PM),https://t.me/indiacovid/12689,NaN,NaN,NaN,NaN,NaN,NaN
3,218994,NaN,07/08/2020,NaN,NaN,NaN,Hyderabad,Telangana,TG,532.0,Hospitalized,NaN,As of 06/08/2020 (8 PM),https://t.me/indiacovid/12689,NaN,NaN,NaN,NaN,NaN,NaN
4,218995,NaN,07/08/2020,NaN,NaN,NaN,Jagtial,Telangana,TG,36.0,Hospitalized,NaN,As of 06/08/2020 (8 PM),https://t.me/indiacovid/12689,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21082,240075,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21083,240076,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21084,240077,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21085,240078,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [113]:
df3 = pd.read_csv("../statewise_tested_numbers_data.csv")

In [115]:
df3[(df3.State == "Maharashtra")]

,Updated On,State,Total Tested,Tag (Total Tested),Positive,Negative,Unconfirmed,Cumulative People In Quarantine,Total People Currently in Quarantine,Tag (People in Quarantine),...,Corona Enquiry Calls,Num Calls State Helpline,Source1,Source2,Source3,Test positivity rate,Tests per thousand,Tests per million,Tests per positive case,Population NCP 2019 Projection
2427,05/04/2020,Maharashtra,16008.0,Tested,NaN,14837,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.00%,0.13,131.0,NaN,122153000.0
2428,06/04/2020,Maharashtra,17563.0,Tested,868.0,15808,NaN,NaN,NaN,NaN,...,NaN,NaN,https://www.cnbctv18.com/healthcare/maharashtr...,NaN,NaN,4.94%,0.14,144.0,20,122153000.0
2429,07/04/2020,Maharashtra,20877.0,Tested,1018.0,19290,569.0,NaN,NaN,NaN,...,NaN,NaN,https://arogya.maharashtra.gov.in/pdf/covidepr...,NaN,NaN,4.88%,0.17,171.0,21,122153000.0
2430,09/04/2020,Maharashtra,20877.0,Tested,868.0,19290,715.0,NaN,NaN,NaN,...,NaN,NaN,https://twitter.com/Maha_MEDD/status/124815689...,https://drive.google.com/file/d/1InVpTI2sfv8N2...,NaN,4.16%,0.17,171.0,24,122153000.0
2431,10/04/2020,Maharashtra,30000.0,Tested,1135.0,28865,NaN,NaN,NaN,NaN,...,NaN,NaN,https://twitter.com/Maha_MEDD/status/124848453...,https://drive.google.com/file/d/1EnHn3WjLO2EkL...,NaN,3.78%,0.25,246.0,26,122153000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2558,16/08/2020,Maharashtra,3165550.0,Tested,607631.0,2557919,NaN,NaN,NaN,NaN,...,NaN,NaN,https://t.co/Gt6lfim8A8?amp=1,https://twitter.com/Maha_MEDD/status/129487943...,NaN,NaN,NaN,NaN,NaN,NaN
2559,17/08/2020,Maharashtra,3208735.0,Tested,NaN,2592249,NaN,NaN,NaN,NaN,...,NaN,NaN,https://t.co/k3aDMnlRNi?amp=1,https://twitter.com/Maha_MEDD/status/129526539...,NaN,NaN,NaN,NaN,NaN,NaN
2560,18/08/2020,Maharashtra,3267625.0,Tested,NaN,2641156,NaN,NaN,NaN,NaN,...,NaN,NaN,https://t.co/mvl9RGXxYQ?amp=1,https://twitter.com/Maha_MEDD/status/129561210...,NaN,NaN,NaN,NaN,NaN,NaN
2561,19/08/2020,Maharashtra,3343052.0,Tested,NaN,2701358,NaN,NaN,NaN,NaN,...,NaN,NaN,https://t.co/6hAQZPlWlv?amp=1,https://twitter.com/Maha_MEDD/status/129598946...,NaN,NaN,NaN,NaN,NaN,NaN


### Method 1:


SoW states Method 1 as:

_To determine a downward trajectory, data are assessed using a smoothed curve. 
To calculate this curve, a cubic spline, or “smoothed curve” is applied, a statistical method that smooths out day-to-day variability in the data. It would use a 7-day smoothened curve over a period of 21 days. The slope of this curve is used to assess declining incidence.
The cubic spline would be used on 4 parameters:_

•	_Active cases_

•	_Fatalities_

•	_Hospital Admissions_

•	_Recovery rates_

**Daily active cases, fatalities, recoveries can be obtained from a combination of districts.csv(for past data) and daily_district_data API for latest data**

**API does not seem to have Hospital admissions per district.**

### Method 2

*Use of the Levitt Metric on Total Deaths to understand past trajectory.
About the Levitt’s Metric:*

*Computed as H(t) = X(t)/X(t-1), where
X(t) is cumulative count until day t, of Covid-19 deaths*


**Total daily deaths is the only metric required by Method2, which can be obtained with a similar combination as that of Method 1**

### Method 3

*21 day Moving Averages for daily cases growth %, Test Positivity Rate % and Daily Tests performed*

**TBD**